In [12]:
# This note is the implementation of past research, sometimes different metric needs different format (JPG/PNG)

from PIL import Image
import base64
import cv2
import csv
from io import StringIO
from io import BytesIO
import numpy as np
from skimage import util, color
import math

In [13]:
infiles = [
#'999*ted.com',
'0*theboneandjointcenter.com',
#'1*mozilla.org',
'2*two-n.com',
#'3*segmentfault.com',
#'4*makeitok.org',
'5*disney.co.jp',
'6*wikipedia.org',
'7*news.yahoo.co.jp',
'8*huxiu.com',
#'9*fangdd.com',
'10*cheshi.com',
'11*humblebundle.com',
'12*theatlantic.com',
#'13*superprof.fr',
'14*microsoft.com',
'15*opera.com',
'16*labinthewild.org',
#'17*nounplus.net',
'18*richyli.com',
'19*pxtoem.com', 
#'20*goofy.photo',
'21*javadrive.jp',
#'22*jcodecraeer.com',
#'23*hdpfans.com',
'24*jiqimao.tv',
'25*clamav.net',
'26*bootcdn.cn',
'27*runoob.com',
'28*tensorfly.cn',
'29*journaldugeek.com', 
'30*matetranslate.com',
'31*kameisyouten.ocnk.net',
'32*gingerweb.jp',
'33*cp.pocky.jp',
'34*aladdin-aic.com',
#'35*ho-ginza.net',
#'36*pandayori.com',
#'37*sekimoto.dental',
'38*kokage-m.com',
#'39*coming-saji.com',
#'40*bluewood.bitter.jp',
'41*steakland.jp',
'42*showroomprive.com',
'43*imas-cg.net',
'44*filetender.com',
'45*hexo.io',
'46*yinwang.org',
'47*blog.yitianshijie.net',
'48*yatani.jp',
'49*qiita.com',
'50*52nlp.cn',
'51*guidetojapanese.org',
'52*olderadults.mobi',
'53*blog.whatsapp.com',
#'54*stratechery.com',
'55*lomake.fi',
#'56*theclinic.cl',
'57*bgmaimuna.com',
'58*0dt.net',
'59*web.ics.purdue.edu',
'60*canon-foundation.jp',
#'61*docs.opencv.org',
#'62*interaction-design.org',
#'63*jlpt.jp',
'64*blog.sciencenet.cn',
'65*pantone.com',
#'66*sdtech.co.jp',
'67*news.livedoor.com',
'68*gmo.jp',
'69*tokai-tv.com', 
'70*life-is-tech.com',
'71*bloomberg.co.jp',
'72*cerezo.jp',
'73*tech.nikkeibp.co.jp',
'74*jp.techcrunch.com',
'75*capcom.co.jp',
#'76*kenkun-jinja.org',
'77*sankei.com',
'78*tech-jp.co.jp',
'79*tech-camp.in',
'80*hasegawa-heart.com',
'81*techplay.jp',
#'82*ubejinja.or.jp',
#'83*19lou.com',
#'84*jiankang.com',
'85*cps.com.cn',
#'86*hea.cn',
#'87*jfc.or.jp',
'88*trafst.jp',
'89*infoq.com',
#'90*secure.j-bus.co.jp',
'91*jcr.incites.thomsonreuters.com',
'92*dna.fr',
'93*macg.co',
'94*cvpr2018.thecvf.com',
'95*chi2019.acm.org',
'96*swellnet.com',
'97*klex.ru',
#'98*kudago.com',
'99*theborneopost.com'
]

In [14]:
# JPG
def edge_density(b64):
    b64 = base64.b64decode(b64)
    b64 = BytesIO(b64)
    img = Image.open(b64)
    img= np.array(img)
    img_la = color.rgb2gray(img)
    img_la = util.img_as_ubyte(img_la)

    # 0.11 and 0.27, sigma = 1,     from Measuring visual clutter
    # See sigma here: https://dsp.stackexchange.com/questions/4716/differences-between-opencv-canny-and-matlab-canny
    img_la = cv2.GaussianBlur(img_la, (7, 7), 1)
    cd = cv2.Canny(img_la, 0.11, 0.27)
    total = cd.shape[0] * cd.shape[1] # Total number of pixels
    number_edges = np.count_nonzero(cd) # Number of edge pixels
    contour_density = float(number_edges) / float(total) # Ratio

    result = contour_density

    return result

In [8]:
# PNG
def edge_congestion(b64):
    b64 = base64.b64decode(b64)
    b64 = BytesIO(b64)
    img = Image.open(b64)
    img = np.array(img)
    img = util.img_as_ubyte(img)
    height, width, depth = img.shape
    borders = np.zeros((img.shape[0], img.shape[1]), dtype=np.int)

    # Do edge detection. create_border return 0 or 255 depending on the difference with neigboring pixels
    for x in range(1, width - 1):
        for y in range(1, height - 1):
            borders[y][x] = create_border(img, borders, y, x)

    count_edge = 0
    count_uncongested = 0
    threshold = 4 # Paper says 20, this is insane. The amount of pixels a person needs to differentiate between two elements

    # Create numpy array from list
    borders = np.array(borders)

    # Assumme screen border is always a border
    for x in range(threshold, width - threshold):
        for y in range(threshold, height - threshold):
            if borders[y][x] == 255:
                count_edge += 1

                # Sum left, right, up, down for number of pixels in threshold
                arr_right = borders[y, x + 1:x + threshold]
                sum_right = sum(arr_right)
                arr_left = borders[y, x - threshold:x - 1]
                sum_left = sum(arr_left)
                arr_up = borders[y + 1:y + threshold, x]
                sum_up = sum(arr_up)
                arr_down = borders[y - threshold:y - 1, x]
                sum_down = sum(arr_down)

                # If the sum is zero, it means there are no other pixels nearby. It needs to be in all directions non-0
                # for a pixel to be congested
                if sum_right == 0 or sum_left == 0 or sum_up == 0 or sum_down == 0:
                    count_uncongested += 1

    try:
        count_congested = count_edge - count_uncongested
        result = float(count_congested) / float(count_edge)
    except ZeroDivisionError:
        result = 0

    return result


def create_border(img, borders, y, x):
    r1 = int(img[y][x][0])
    g1 = int(img[y][x][1])
    b1 = int(img[y][x][2])

    points_2 = [
        [y, x + 1],
        [y, x - 1],
        [y + 1, x],
        [y - 1, x]
    ]

    ret = 0
    for n in range(4):
        x2 = points_2[n][1]
        y2 = points_2[n][0]

        r2 = int(img[y2][x2][0])
        g2 = int(img[y2][x2][1])
        b2 = int(img[y2][x2][2])

        dst_r = math.fabs(r2 - r1)
        dst_g = math.fabs(g2 - g1)
        dst_b = math.fabs(b2 - b1)

        if (dst_r > 50 or dst_b > 50 or dst_g > 50) and borders[y2][x2] == 0:
            ret = 255
            break

    return ret


In [3]:
# JPG
def jpeg_file_size_metric(b64):
    jpg_size = int(len(b64) * 0.75)   
    return jpg_size

In [9]:
# PNG
def figure_ground_contrast(b64):
    b64 = base64.b64decode(b64)
    b64 = BytesIO(b64)
    img = Image.open(b64)
    img= np.array(img)
    img_la = color.rgb2gray(img)
    img_la = util.img_as_ubyte(img_la)

    # Get the number of edge pixels per level. See 1
    edge_per_level = []
    for x in range(1, 8):
        # Blur is needed: https://dsp.stackexchange.com/questions/4716/differences-between-opencv-canny-and-matlab-canny
        img_la = cv2.GaussianBlur(img_la, (7, 7), 2)
        cd = cv2.Canny(img_la, x * 0.04, x * 0.1) # Higher level from 0.1-0.7, lower level is 40% of higher
        number_edges = np.count_nonzero(cd) # Number of edge pixels
        edge_per_level.append(number_edges)

    difference = []
    
    # Calculate the difference between each level
    for x in range(len(edge_per_level) - 1):
        difference.append(edge_per_level[x] - edge_per_level[x + 1])

    # Give weight per level. Lower levels have more impact so higher weight
    weighted_sum = 0
    for x in range(len(difference)):
        weighted_sum += difference[x] * (1.0 - ((x - 1.0) / 6.0))

    # Normalize
    try:
        result = weighted_sum / (edge_per_level[0] - edge_per_level[5])
    except ZeroDivisionError:
        result = 0

    return result

In [18]:
# PNG
def get_pixels_in_radius(x, y, width, height, radius):
    # Get x border
    if x < radius:
        rad_x_left = -x
        rad_x_right = radius
    elif width - x < radius:
        rad_x_right = 1 * (width - x)
        rad_x_left = -radius
    else:
        rad_x_left = -radius
        rad_x_right = radius

    # Get y borders
    if y < radius:
        rad_y_top = -y
        rad_y_bottom = radius
    elif height - y < radius:
        rad_y_bottom = 1 * (height - y)
        rad_y_top = -radius
    else:
        rad_y_top = -radius
        rad_y_bottom = radius

    pixels = []
    for m in range(rad_x_left, rad_x_right):
        for n in range(rad_y_top, rad_y_bottom):
            if m != 0 or n != 0:
                pixel = [x + m, y + n]
                pixels.append(pixel)

    return pixels


def pixel_symmetry(b64):
    b64 = base64.b64decode(b64)
    b64 = BytesIO(b64)
    img = Image.open(b64)
    img = np.array(img)
    img_la = color.rgb2gray(img)
    img_la = util.img_as_ubyte(img_la)

    # See sigma here: https://dsp.stackexchange.com/questions/4716/differences-between-opencv-canny-and-matlab-canny
    img_la = cv2.GaussianBlur(img_la, (7, 7), 2)
    edges = cv2.Canny(img_la, 0.11, 0.27)

    height, width = edges.shape

    # Set all pixels in radius of an edge pixel to 0
    # This is not going good yet
    radius = 3
    all_key = 0
    for y in range(height):
        for x in range(width):
            if edges[y][x] != 0:
                all_key += 1
                pixels_in_radius = get_pixels_in_radius(x, y, width, height, radius)
                for pixel in pixels_in_radius:
                    edges[pixel[1], pixel[0]] = 0

    img = Image.fromarray(edges, 'L')
    # img.show()
    symmetry_radius = 4
    
    # Check vertical symmetry
    sym_key = 0
    for y in range(height):
        for x in range(int(width / 2)):
            if edges[y][x] != 0:
                vertical_pixels = get_pixels_in_radius(width - x, y, width, height, symmetry_radius)
                horizontal_pixels = get_pixels_in_radius(x, height - y, width, height, symmetry_radius)

                for pixel in vertical_pixels:
                    if edges[int(pixel[1]), int(pixel[0])] != 0:
                        sym_key += 1
                        break

                for pixel in horizontal_pixels:
                    if edges[int(pixel[1]), int(pixel[0])] != 0:
                        sym_key += 1
                        break

    try:
        sym_normalized = (float(sym_key) / float(all_key)) * ((float((all_key - 1) * symmetry_radius) / float(width * height)) ** -1)
    except ZeroDivisionError:
        sym_normalized = 0

    return sym_normalized
    

In [7]:
# PNG
def png_size(b64):
    png_size = int(len(b64) * 0.75)
    return png_size

In [15]:
# JPG
jpeg_file_size_list = []
edge_density_list = []

# PNG
edge_congestion_list = []
figure_ground_contrast_list = []
pixel_symmetry_list = []
png_size_list = []


def main():
    for i in infiles:
        no, title = i.split('*')
#         with open('webpages/'+ title +'.png', 'rb') as f:
        with open('webpagesJ/'+ title +'.jpg', 'rb') as f:
            im = f.read()
            b64 = base64.b64encode(im)
            
        result = edge_density(b64)
        
        thisItem = [title, result]
        print(thisItem)
        
        edge_density_list.append(thisItem)

    with open('edge_density.csv', 'w') as  f:
        writer = csv.writer(f)
        writer.writerow(['title', 'edge_density'])
        writer.writerows(edge_density_list)

In [16]:
main()

/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/skimage/util/dtype.py:141: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


['theboneandjointcenter.com', 0.06851578947368421]
['two-n.com', 0.052961988304093564]
['disney.co.jp', 0.09489181286549708]
['wikipedia.org', 0.0320046783625731]
['news.yahoo.co.jp', 0.06599941520467836]
['huxiu.com', 0.07880701754385964]
['cheshi.com', 0.1054345029239766]
['humblebundle.com', 0.14866666666666667]
['theatlantic.com', 0.08849005847953216]
['microsoft.com', 0.12703099415204677]
['opera.com', 0.0391374269005848]
['labinthewild.org', 0.06529824561403509]
['richyli.com', 0.05052280701754386]
['pxtoem.com', 0.05669181286549708]
['javadrive.jp', 0.04950701754385965]
['jiqimao.tv', 0.12799824561403508]
['clamav.net', 0.043761988304093564]
['bootcdn.cn', 0.041501169590643275]
['runoob.com', 0.0644701754385965]
['tensorfly.cn', 0.14785964912280702]
['journaldugeek.com', 0.09976198830409357]
['matetranslate.com', 0.09841637426900585]
['kameisyouten.ocnk.net', 0.21593859649122807]
['gingerweb.jp', 0.08474444444444444]
['cp.pocky.jp', 0.07041637426900585]
['aladdin-aic.com', 0.253